# Welcome to Express Train *🚂*

<img src="https://github.com/asatriano/expresstrain/blob/main/images/express_train_logo.png?raw=true" width=“800” height=“200” />

<!-- Express Train Tutorial -->

What we're learning today:


*   How to use Express Train to eliminate the boiler plate
*   How can we introduce easily in the loop anything we want. Express Train is super flexible, and today we'll be making 3 easy examples:
    - Automated Mixed Precision (FP16)
    - Gradient Accumulation
    - using learning rate schedulers (as an example of hooks usage)

We'll do so by adapting the [native pytorch MNIST example](https://github.com/pytorch/examples/blob/master/mnist/main.py) to Express Train.

For more on Express Train, visit, clone or star our [Github project](https://github.com/asatriano/expresstrain). 🙈




# Let's start :)

We'll be making some basic imports:

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import os

We'll clone latest version of Express Train by cloning it from our [Github project](https://github.com/asatriano/expresstrain)

In [2]:
os.system("rm -rf ./expresstrain") # to make sure you're up to date :)
os.system("git clone https://github.com/asatriano/expresstrain")

0

Let's import Express Train:

In [3]:
import expresstrain as et

Subclass Express Train to your heart's content:

For all available hooks (documentation to come, and more hooks to come), visit our [Github project](https://github.com/asatriano/expresstrain)







In [4]:
# Subclass Express Train:
class CustomExpressTrain(et.ExpressTrain):
    def __init__(self, **kwargs):
        super(CustomExpressTrain, self).__init__()
        self.initialize_all(kwargs)

    

    def on_train_epoch_end(self):
        self.scheduler_every_epoch.step()

Let's define our mighty Model:

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x 
    #logits (or compute logsoftmax, and use a NLLLoss as a 
    # custom loss function in ExpressTrainer)

Let's define some metrics we will use later:

In [6]:
def accuracy(preds, targets):
    assert(len(preds)==len(targets))
    correct=torch.sum(preds == targets)
    return correct/len(targets)

Our parameters.

Go ahead and experiment by:
*   toggling `use_fp16` as `True` or `False`
*   changing `backward_every` to a value higher than `1` to activate gradient accumulation 




In [7]:
# input random seed integer (default: 42) (type=int):
random_seed=42
# input batch size to use at training (default: 32) (type=int):
batch_size=32
# input batch size multiplier for validation (default: 2) (type=int):
batch_size_multiplier=2
# input number of workers for dataloaders (default: 0) (type=int):
num_workers_dataloader=0
# input training learnign rate (default: 3e-4) (type=float):
learning_rate=1e-2
# Learning rate step size (default: 2) (type=int)
step_size_lr_scheduler=2
# Learning rate step gamma (default: 0.7) (type=float)
gamma=0.7
# input training epochs (default=10) (type=int):
epochs=30
# input saving path for loss and metrics (default: None) (type=str):
path_performance=None
# input saving path for loss, metric, and model params (default: None) (type=str):
path_perf_model=None
# input whether to use Automatic Mixed Precision (default: True) (type=bool):
use_fp16=False
# input how many batches between backward passes (default: 1) (type=bool):
backward_every=1

Define your device:

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device used: {device}")

torch.manual_seed(random_seed)

Device used: cuda


Define your data transforms:

In [9]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

Import your datasets:

In [10]:
dataset1 = datasets.FashionMNIST('./data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.FashionMNIST('./data', train=False,
                    transform=transform)

Define your data loaders:

In [11]:
assert(batch_size>backward_every)
train_kwargs = {'batch_size': batch_size//backward_every,
                'shuffle': True}
valid_kwargs = {'batch_size': batch_size*batch_size_multiplier//backward_every,
                'shuffle': False}
workers_kwargs = {'num_workers': num_workers_dataloader}

train_kwargs.update(workers_kwargs)
valid_kwargs.update(workers_kwargs)
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
valid_loader = torch.utils.data.DataLoader(dataset2, **valid_kwargs)

Instance your model and optimizer:

In [12]:
model = Net().to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

Setup your LR scheduler

In [13]:
scheduler_kwargs={'step_size': step_size_lr_scheduler,
                'gamma': gamma}
scheduler_every_epoch = torch.optim.lr_scheduler.StepLR(optimizer, **scheduler_kwargs)

# Use your Custom Express Train trainer:

1/2: Instance your custom Express Train trainer:

In [14]:
# Instance your Custom Express Train trainer
trainer_kwargs={'train_loader': train_loader,
                'valid_loader': valid_loader,
                'model': model,
                'num_classes': 10,
                'device': device,
                'learning_rate': learning_rate,
                'optimizer': optimizer,
                'scheduler_every_epoch': scheduler_every_epoch,
                'metric_used': accuracy,
                'path_performance': path_performance,
                'path_performance_and_model': path_perf_model,
                'backward_every': backward_every}
if use_fp16==True:
    print("Using Automatic Mixed Precision")
    trainer_kwargs.update({'fp16': use_fp16})

trainer=CustomExpressTrain(**trainer_kwargs)

2/2 Fit! 🥰🚂

In [15]:
trainer.fit(epochs)

Epoch 1/30, accuracy_train: 77.73
Epoch 1/30, accuracy_valid: 83.68
Epoch 2/30, accuracy_train: 80.45
Epoch 2/30, accuracy_valid: 84.59
Epoch 3/30, accuracy_train: 82.68
Epoch 3/30, accuracy_valid: 83.64
Epoch 4/30, accuracy_train: 83.16
Epoch 4/30, accuracy_valid: 85.13
Epoch 5/30, accuracy_train: 84.43
Epoch 5/30, accuracy_valid: 87.71
Epoch 6/30, accuracy_train: 84.67
Epoch 6/30, accuracy_valid: 87.34
Epoch 7/30, accuracy_train: 85.60
Epoch 7/30, accuracy_valid: 87.34
Epoch 8/30, accuracy_train: 85.78
Epoch 8/30, accuracy_valid: 87.65
Epoch 9/30, accuracy_train: 86.57
Epoch 9/30, accuracy_valid: 88.68
Epoch 10/30, accuracy_train: 86.85
Epoch 10/30, accuracy_valid: 88.22
Epoch 11/30, accuracy_train: 87.29
Epoch 11/30, accuracy_valid: 89.01
Epoch 12/30, accuracy_train: 87.58
Epoch 12/30, accuracy_valid: 89.04
Epoch 13/30, accuracy_train: 88.09
Epoch 13/30, accuracy_valid: 89.25
Epoch 14/30, accuracy_train: 88.29
Epoch 14/30, accuracy_valid: 89.07
Epoch 15/30, accuracy_train: 88.57
Epo

# You're done! 🎉

See what other methods you can use to customize your loops, inside of our [Express Train model definition](https://github.com/asatriano/expresstrain/blob/main/expresstrain/model.py)

Start [Express Train on GitHub](https://github.com/asatriano/expresstrain)

Please provide all feedback directly on [Github](https://github.com/asatriano/expresstrain) 😊